In [3]:
import torch
import torch.nn as nn
from torchvision import models

In [18]:

# Load pretrained ResNet model
resnet = models.resnet50(pretrained=True)


d:\D-Program Files\Python\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\D-Program Files\Python\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [19]:
list(resnet.children())

[Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False),
 BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 ReLU(inplace=True),
 MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False),
 Sequential(
   (0): Bottleneck(
     (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
     (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
     (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (relu): ReLU(inplace=True)
     (downsample): Sequential(
       (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
       (1): BatchNorm2d(256, eps=1e-05, momentum

In [15]:

# Remove the fully connected layer and the pooling layer
# Keep layers up to the penultimate layer
feature_extractor = nn.Sequential(*list(resnet.children())[:-2])


In [16]:
# Test the feature extractor
input_tensor = torch.randn(1, 3, 224, 224)  # Example input (batch_size=1, 3 channels, 224x224 image)
features = feature_extractor(input_tensor)
print("Feature shape:", features.shape)

Feature shape: torch.Size([1, 2048, 7, 7])


In [22]:
import os
import torch
import numpy as np
from torchvision import models, transforms
from PIL import Image


In [23]:
input_folder = "D:/D-Documents/Self-Improvement/Python/Computer Vision/MS-TCN/Multi-Model/data/gtea/frames"
output_folder = "D:/D-Documents/Self-Improvement/Python/Computer Vision/MS-TCN/Multi-Model/data/gtea/extracted_frame_features"

In [24]:

# Create output directory if not exists
os.makedirs(output_folder, exist_ok=True)

# Load ResNet model for feature extraction
resnet = models.resnet50(pretrained=True)
feature_extractor = torch.nn.Sequential(*list(resnet.children())[:-1])  # Remove the fully connected layer
feature_extractor.eval()

# Define preprocessing transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Process each video folder
for video_folder in os.listdir(input_folder):
    video_path = os.path.join(input_folder, video_folder)
    if not os.path.isdir(video_path):
        continue  # Skip if it's not a folder

    # Create a corresponding folder in the output directory
    video_output_path = os.path.join(output_folder, video_folder)
    os.makedirs(video_output_path, exist_ok=True)

    print(f"Processing video folder: {video_folder}")

    # Process each frame
    for frame_file in os.listdir(video_path):
        frame_path = os.path.join(video_path, frame_file)

        # Check for valid image file
        if not (frame_file.endswith(".png") or frame_file.endswith(".jpg")):
            continue

        # Load and preprocess frame
        image = Image.open(frame_path).convert("RGB")
        input_tensor = transform(image).unsqueeze(0)  # Add batch dimension

        # Extract features
        with torch.no_grad():
            features = feature_extractor(input_tensor).squeeze().numpy()  # Remove batch and spatial dimensions

        # Save features as .npy
        feature_file = os.path.splitext(frame_file)[0] + ".npy"
        feature_path = os.path.join(video_output_path, feature_file)
        np.save(feature_path, features)

    print(f"Features saved for video folder: {video_folder}")

print("Feature extraction completed!")


d:\D-Program Files\Python\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\D-Program Files\Python\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Processing video folder: S1_Cheese_C1
Features saved for video folder: S1_Cheese_C1
Processing video folder: S1_CofHoney_C1
Features saved for video folder: S1_CofHoney_C1
Processing video folder: S1_Hotdog_C1
Features saved for video folder: S1_Hotdog_C1
Processing video folder: S1_Pealate_C1
Features saved for video folder: S1_Pealate_C1
Processing video folder: S1_Peanut_C1
Features saved for video folder: S1_Peanut_C1
Processing video folder: S1_Tea_C1
Features saved for video folder: S1_Tea_C1
Processing video folder: S2_Cheese_C1
Features saved for video folder: S2_Cheese_C1
Processing video folder: S2_Coffee_C1
Features saved for video folder: S2_Coffee_C1
Processing video folder: S2_CofHoney_C1
Features saved for video folder: S2_CofHoney_C1
Processing video folder: S2_Hotdog_C1
Features saved for video folder: S2_Hotdog_C1
Processing video folder: S2_Pealate_C1
Features saved for video folder: S2_Pealate_C1
Processing video folder: S2_Peanut_C1
Features saved for video folder: